# `unittest`

```{warning}
`unittest` a Jupyter-Book si moc dobře nerozumí - výstupy pod některými testy jsou podivné/rozbité. Než to opravím, doporučuji si to vyzkoušet ručně.
```

Základními stavebními kameny balíku `unittest` jsou čtyři koncepty:

- *test fixture* (někdy test context) reprezentuje přípravu všeho, co je na daný test třeba (načtení dat, připojení, příprava directory structure)
- *test case* testovací jednotka. Typicky určena pro testování jednoho kusu kódu (unit), obsahuje řadu konkrétních testů.
- *test suite* agreguje testy do větších celků
- *test runner* zajišťuje spouštění testů.

Začneme jednoduchou funkcí:

In [7]:
def add(x, y):
    return x + y

Balík `unittest` dělí testy do jednotlivých *test cases*, které definujeme jako potomka třídy `unittest.TestCase`. V jednom `TestCase` poté můžeme definovat libovolné množství metod. Metody, jejichž název má prefix `test_` pak reprezentují jednotlivé testy. Jeden `TestCase` tak může obsahovat více testů. Typicky se do `TestCase` snažím sdružit nějak příbuzné testy, tj. např. testy jednoho objektu, funkce apod.

Klíčovým prvkem každého testu je kontrola, zda testovaný kód dává správný výstup. K takovým kontrolám typicky slouží funkce typu `assert`, které zkontrolují, zda je jejich argument pravdivý. Není-li, vyvolají výjimku typu `AssertionError`. Hle:

In [ ]:
try:
    assert(1 == 2)
except AssertionError:
    print("It failed, because 1 is not equal to 2")

Balík `unittest` přináší ve třídě `unittest.TestCase` řadu metod, které implementují `assert` pro specifické případy (např. porovnávání floatů, kolekcí atd.). Použití těchto metod je výhodné - šetří práci, protože už za nás řeší logiku složitějších asserts.

```{note}
Protože knihu kompiluji pomocí Jupyter-Book, musím k ukázkám testů přidávat `unittest.main(argv=[''], exit=False)`. Za normálních okolností pro stadardní spouštění testů to není třeba.
```

In [ ]:
import unittest

class TestAdd(unittest.TestCase):
    def test_add(self):
        self.assertEqual(add(1, 2), 3)

unittest.main(defaultTest="TestAdd", argv=[''], exit=False)

V praxi je často vhodné otestovat více vstupních hodnoty, ale dělat to manuálně je trochu krkolomné:

In [ ]:
import unittest

class TestAdd(unittest.TestCase):
    def test_add(self):
        self.assertEqual(add(1, 2), 3)
        self.assertEqual(add(2, 1), 4)
        print("tady")
        self.assertEqual(add(-1, 2), 1)

unittest.main(defaultTest="TestAdd", argv=[''], exit=False)

Pro parametrizované testy tak používáme metodu `.subTest`, jako v následujícím příkladě. Subtesty mají oproti předchozí možnosti i tu výhodu, že když jeden selže, proběhnou i ostatní a neselže tak celá metoda `test_add`. Přidal jsem několik záměrně vadných testů, které to ilustrují.

In [ ]:
import unittest
from itertools import product

class TestAdd(unittest.TestCase):
    def test_add(self):
        for a, b in product(range(5), range(6)):
            with self.subTest(a=a, b=b):
                if a == 1 and b >= 4:
                    self.assertEqual(add(a, b), a)
                else:
                    self.assertEqual(add(a, b), a+b)


unittest.main(defaultTest="TestAdd", argv=[''], exit=False)

Následující `TestCase` obsahuje více testů. Všimněte si volby správné assert metody pro floaty.

In [ ]:
import unittest

def add(x, y):
    return x + y

class TestAdd(unittest.TestCase):
    def test_add_int(self):
        self.assertEqual(add(1, 2), 3)

    def test_add_str(self):
        self.assertEqual(add("a", "b"), "ab")

    def test_add_float(self):
        self.assertAlmostEqual(add(0.1, 0.2), 0.3)

unittest.main(defaultTest="TestAdd", argv=[''], exit=False)

## Test discovery, organizace, pokrytí

Testy obyvkle umísťujeme do složky `tests` vedle balíčku. Tedy např. ukázkový balíček `mipy` z hodiny o struktuře balíku má strukturu:

```bash
mipy
|-- mipy
|    |-- math
|    |    |-- __init__.py
|    |    |-- other.py
|    |    |-- primes.py
|    |-- __init__.py
|    |-- __main__.py
|    |-- utils.py
|-- tests
|    |-- __init__.py
|    |-- test_math_other.py
|    |-- test_primes.py
|    |-- test_utils.py
|-- pyproject.toml
```
Testy členíme do souborů s prefixem `test_` a obvykle držíme strukturu jeden soubor na jeden modul v balíku (není to ovšem nijak závazné). Každý soubor může obsahovat více `TestCase`. Složka `tests` musí obsahovat `__init__.py`, tedy musí fungovat jako balíček.

Nejsnazší způsob jak spustit testy je ze složky `mipy` příkazem
```bash
python3 -m unittest
```
Python nejprve spustí proces zvaný *test discovery*, který projde rekurzivně aktuální složku a vyhledá všechny testy, sestaví je do takzvaného  `TestSuite`, tedy jakési kolekce testů, kterou dále předá objektu typu `TestRunner`, který je zodpovědný za jejich spouštění. Testy lze do `TestSuite` přidávat i manuálně, ale o tom jindy.

Balík `unittest` má mnoho parametrů a možností, jak testy spouštět. Občas je užitečné nechat podrobně vypsat, co se zrovna děje (přepínač `-v` nebo `--verbose`), případně spouštět jen některý test. Například:
```bash
$ python3 -m unittest -v tests/test_primes.py

test_called (tests.test_primes.TestPrimes) ... ok
test_primes (tests.test_primes.TestPrimes) ... ok
test_some (tests.test_primes.TestPrimes) ... ok
test_zero (tests.test_primes.TestPrimes) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.001s

OK
```

Většina IDE umí testy detekovat, spouště a zpracovávat jejich výstup, díky čemuž je používání testů velmi komfortní.


## Setup a Teardown

Některé testy ze své podstaty vyžadují data (například proto, že testovaná funkce s těmi daty nějak pracuje). Bývá proto praktické přípravu takových data oddělit od samotného testu a případně zajistit, aby se data nenačítala zbytečně dvakrát, zejména v případech, kdy jich je větší množství nebo je jejich příprava obecně pomalá.

K tomu slouží metody `.setUp` a `.tearDown` definované na objektu typu `TestCase`, které se volají před, respektive po, vykonání jakýchkoliv testů v daném `TestCase`.

Ukažme si to napříkladu jednoduchých statistických funkcí.

In [ ]:
import unittest

def mean(x):
    return sum(x) / len(x)

def median(x):
    i = len(x) // 2 - 1

    if len(x) % 2 == 0:
        return (x[i] + x[i+1]) / 2
    else:
        return x[i]
    

class TestStat(unittest.TestCase):
    def setUp(self):
        self.data = list(range(1, 101))

    def test_mean(self):
        self.assertAlmostEqual(mean(self.data), 50.5)
    
    def test_empty(self):
        self.assertRaises(ZeroDivisionError, mean, [])

    def test_median(self):
        self.assertAlmostEqual(median(self.data), 50.5)


unittest.main(defaultTest="TestStat", argv=[''], exit=False)

Obecně může být v těchto metodách jakákoliv příprava na testy: vytváření dočasných souborů a jejich následná likvidace, připojení a odpojení databáze atd.

## Mocking and patching

Mocking je velice užitečnou technikou sloužící k nahrazení některých kusů codebase falešnými objekty (mock objects), což umožňuje izolované testování nezávislé na nahrazovaných objektech. Konkrétně pojem *mocking* označuje vytváření toho falešného, zástupného objektu, zatímco *patching* značí jeho dynamické podstrčení za objekt skutečný. 

V balíčku `unittest` je k disposici objekt typu `Mock`, kterému můžeme podstrčit metody napodobovaného objektu a posléze kontrolovat, zda byly zavolány se správnými parametry atp.

Nejjednodušší je asi příklad.

In [22]:
class Logger:
    def log(self, message):
        print(f"Logging: {message}")

class Calculator:
    def __init__(self):
        self.logger = Logger()

    def add(self, a, b):
        result = a + b
        self.logger.log(f"Added {a} to {b} got {result}")
        return result

In [ ]:
import unittest
from unittest.mock import Mock

class TestCalculator(unittest.TestCase):
    def test_addition(self):
        calculator = Calculator()
        calculator.logger = Mock()

        result = calculator.add(1, 2)

        self.assertEqual(result, 3)
        calculator.logger.log.assert_called_once_with("Added 1 to 2 got 3")

unittest.main(defaultTest="TestCalculator", argv=[''], exit=False)

V tomto příkladě jsme nahradili `logger` v objektu `Calculator` mock objektem, který sice neumí nic, co skutečný `Logger`, ale protože testujeme `Calculator`, tak to nevadí. Oproti tomu můžeme zkontrolovat, že `Calculator` se snaží `Logger` používat a že mu předává správné informace. Díky tomu navíc nejsme závislí na side effects objektu `Logger` (např. na vytváření nějakých logfiles apod.), takže nemusíme nic uklízet, nebo na tom, zda `Logger` funguje. Testování `Calculator` je tak izolováno od `Logger`.

Pokud `Mock` objekt není zavolán, unittest samozřejmě selže:

In [ ]:
from unittest.mock import Mock

mock = Mock(return_value=4)
result = mock(1, 2, 3)

try:
    mock.assert_called_with(1, 2)
except AssertionError as e:
    print(e)

Mock objektům můžeme podstrčit i návratové hodnoty. To je praktické např. v případech, kdy testovaná funkce závisí na výstupu z jiných objektů, které ovšem můžou být pomalé, nákladné na chod, nebo jejich chod může být závislý na vnějších faktorech (např. síťová komunikace). Návratová hodnota se ukrývá pod názvem `return_value`:

In [ ]:
from unittest.mock import Mock

mock = Mock()
mock.return_value = 5

mock()

Alternativou je využití atributu `side_effect`, na který lze navázat existující funkci nebo např. výjimku.

In [ ]:
from unittest.mock import Mock

def some_side_effect():
    print("this is a side effect")

mock = Mock()
mock.side_effect = some_side_effect

mock()

In [ ]:
from unittest.mock import Mock

mock = Mock()
mock.side_effect = ZeroDivisionError("a specific Exception as a side effect")

try:
    mock()
except ZeroDivisionError as e:
    print(e)

Balík `unittest` nabízí několik možností jak dynamicky nahradit existující objekt (patching). Zkusme nahradit funkci `math.sqrt` funkcí `fake_sqrt`, která bude vracet dvojnásobek čísla místo skutečné odmocniny.

In [28]:
def fake_sqrt(x):
    return 2 * x

In [ ]:
import math
from unittest.mock import patch

@patch("math.sqrt")
def f(x, mock):
    mock.side_effect = fake_sqrt
    print(math.sqrt(x))

f(100)
print(math.sqrt(100))

In [ ]:
import math
from unittest.mock import patch

with patch("math.sqrt") as mock:
    mock.side_effect = fake_sqrt
    print(math.sqrt(100))

print(math.sqrt(100))


In [ ]:
import math
from unittest.mock import patch

with patch("math.sqrt", side_effect=fake_sqrt):
    print(math.sqrt(100))

print(math.sqrt(100))

```{note}
Balík `datetime` má svá specifika, která můžou komplikovat mocking. Přikládám odkaz na článek, který vysvětluje, co s tím: https://hakibenita.com/python-dependency-injection
```